# Data Gathering, Wrangling and Merging
This notebook contains code and references used to create the dataset used in the `Computer Programming and Data Management Assignment` project.

All code and data relating to the project is available on GitHub: https://github.com/NicoloZorzetto/computer_programming_and_data_management_assignment .

This, along with the practices used below and in the other files (suck as sha256 checksums, dirlistings and automatic download if possible), assures transparency and reproducibility of the work conducted.

License
--------
GPL v3

# HS code data

## Sources and notes

### Tariffs
The DataWeb US platform offers annual data on tariffs by HS commodity code, divided by years and with only zip folders containing the data in excel and txt (comma separated) formats.

Since all direct download links follow the same pattern we can automatically download them in `data/tariffs/raw`, unzip them without writing on disk, open only the `xls` files as pandas dataframes so we can add the year column, append and parse them.

Several variable names differ only due to historical naming conventions (like MFN average rates having year-specific suffixes) so we harmonize them to obtain a consistent set of structural tariff rates.

In [1]:
from pathlib import Path
import wget
import zipfile
import pandas as pd
import hashlib
import os

In [2]:
from pathlib import Path
import zipfile
import pandas as pd

In [3]:
PROJECT_ROOT = Path.cwd().resolve().parent

TARIFFS_DATA_DIR = PROJECT_ROOT / "data" / "tariffs"
TARIFFS_RAW_DIR = TARIFFS_DATA_DIR / "raw"
TARIFFS_OUT_APPENDED_CSV = TARIFFS_DATA_DIR / "hs_annual_tariffs_raw_appended.csv"
TARIFFS_OUT_CLEANED_CSV = TARIFFS_DATA_DIR / "hs_annual_tariffs_cleaned.csv"


TARIFFS_RAW_DIR.mkdir(parents=True, exist_ok=True)
TARIFFS_DATA_DIR.mkdir(parents=True, exist_ok=True)

PROJECT_ROOT, TARIFFS_RAW_DIR, TARIFFS_OUT_APPENDED_CSV

(PosixPath('/home/nicolo/Documents/uni/computer_programming_and_data_management/2/exam/assignment'),
 PosixPath('/home/nicolo/Documents/uni/computer_programming_and_data_management/2/exam/assignment/data/tariffs/raw'),
 PosixPath('/home/nicolo/Documents/uni/computer_programming_and_data_management/2/exam/assignment/data/tariffs/hs_annual_tariffs_raw_appended.csv'))

#### Sha256 function
By checking the existance of the appended file and its sha256 hash (computed after sorting to ensure stability across runs), we avoid re-downloading raw data or re-parsing the files which is time and computationally expensive.

If the hash doesn't correspond - due to the file missing or being manipulated - we check the presence of all raw files, downloading them if needed and then parse them all to re-build the complete dataset.

In [4]:
def dataframe_sha256(df: pd.DataFrame) -> str:
    """Compute sha256 hash of a dataframe after sorting it."""
    normalized = (
        df.sort_index(axis=1)
        .sort_values(by=df.columns.tolist(), kind="mergesort")
    )
    csv_bytes = normalized.to_csv(index=False).encode("utf-8")
    return hashlib.sha256(csv_bytes).hexdigest()

In [5]:
if TARIFFS_OUT_APPENDED_CSV.exists():
    tariffs_raw_df = pd.read_csv(TARIFFS_OUT_APPENDED_CSV, dtype="string")
    current_tariffs_hash = dataframe_sha256(tariffs_raw_df)
else:
    current_tariffs_hash = ''

#### Raw data automatic download

In [6]:
# Link generator
years = list(range(1997, 2026))  # 1997..2025

prefix = "https://www.usitc.gov/tariff_affairs/documents/tariff_data/tariff_data_"
suffix = ".zip"

links = []
for y in years:
    links.append((y, f"{prefix}{y}{suffix}"))

print("Links generated.")

Links generated.


In [7]:
if current_tariffs_hash == '78a98c6ff065bce0ba504c57f6309e7052b42ee66bae167cbd959b54ddbe05ed':
    rebuild_tariffs_data = False
    print(f"All data already exists in its concatenated form in {TARIFFS_OUT_APPENDED_CSV} .")
else:
    rebuild_tariffs_data = True

All data already exists in its concatenated form in /home/nicolo/Documents/uni/computer_programming_and_data_management/2/exam/assignment/data/tariffs/hs_annual_tariffs_raw_appended.csv .


In [8]:
# Data downloader
if rebuild_tariffs_data:
    for year, url in links:
        out = TARIFFS_RAW_DIR / f"tariff_data_{year}.zip"
        if not out.exists():
            # Had to specify user-agent because the server was denying requests
            # !wget --user-agent="Mozilla/5.0 (X11; Linux x86_64)" -O "{out}" "{url}"
            print(f"Downloading {year} data...")
            wget.download(url, out=str(out))
        else:
            print(f"Already exists {year}")

        if out.stat().st_size == 0:
            print(f"[!]  {year}: empty file")

#### Data extraction and concatenation
We extract the excel (`xlsx` files) from the zip archives on-the-fly without writing to disk by opening them as pandas dataframes.

To each we add a column for the year and finally append (`concat`) them all to create a single `year`-`hs code` tariff data panel.

The raw appended data is saved to `data/tariffs/hs_annual_tariffs_raw_appended.csv` before cleaning for auditability and transparency.

In [9]:
# Raw files parser
dfs = []
failed = []

if rebuild_tariffs_data:
    print(f"Parsing raw files...")

    for year in years:
        zip_path = TARIFFS_RAW_DIR / f"tariff_data_{year}.zip"

        try:
            with zipfile.ZipFile(zip_path, "r") as z:
                excel_files = [n for n in z.namelist() if n.lower().endswith((".xlsx"))]
                if len(excel_files) == 0:
                    raise FileNotFoundError(f"No xlsx in {zip_path.name}")

                excel_name = excel_files[0]
                with z.open(excel_name) as f:
                    df = pd.read_excel(f, dtype="string")

            df["year"] = year
            dfs.append(df)

            print(f"[OK] {year}: {df.shape} ({excel_name})")

        except Exception as e:
            failed.append((year, str(e)))
            print(f"[FAILED] {year}: {e}")

    if len(failed) == 0:
        print(f"No parsing failed, appending and saving to {TARIFFS_OUT_APPENDED_CSV} ...")
        tariffs_raw_df = pd.concat(dfs, ignore_index=True, sort=False)
        tariffs_raw_df
        tariffs_raw_df.to_csv(TARIFFS_OUT_APPENDED_CSV, index=False)
    else:
        print("Errors encountered when parsing the following files:")
        print(failed)

#### Data cleaning

##### Column cleanup
Some variables have to be renamed (combined) because of editorial and historical inconsistencies:
- some `mfn_ave` columns have year-specific suffixes
- `colombia` was spelled `columbia` in some files
- column names starting with `col(n)` are structural but confusing, so we give them descriptive names
- we strip any leading or trailing spaces and lowercase all column names for uniformity

In [10]:
tariffs_df = tariffs_raw_df.copy()

In [11]:
tariffs_df.columns.tolist()

['hts8',
 'brief_description',
 'quantity_1_code',
 'quantity_2_code',
 'mfn_ave97',
 'mfn_rate_type_code',
 'wto_binding_code',
 'mfn_ad_val_rate',
 'mfn_specific_rate',
 'mfn_other_rate',
 'gsp_indicator',
 'gsp_ctry_excluded',
 'apta_indicator',
 'civil_air_indicator',
 'cbi_indicator',
 'cbi_ad_val_rate',
 'cbi_specific_rate',
 'israel_fta_indicator',
 'atpa_indicator',
 'atpa_ad_val_rate',
 'atpa_specific_rate',
 'pharmaceutical_ind',
 'dyes_indicator',
 'nafta_canada_ind',
 'canada_ad_val_rate',
 'canada_specific_rate',
 'canada_other_rate',
 'nafta_mexico_ind',
 'mexico_ad_val_rate',
 'mexico_specific_rate',
 'mexico_other_rate',
 'col2_rate_type_code',
 'col2_ad_val_rate',
 'col2_specific_rate',
 'col2_other_rate',
 'begin_effect_date',
 'end_effective_date',
 'year',
 'mfn_ave98',
 'mexico_rate_type_code',
 'mfn_ave99',
 'additional_tariff_num_ind',
 'mfn_ave',
 'agoa_indicator',
 'cbtpa_indicator',
 'cbtpa_rate_type_code',
 'cbtpa_ad_val_rate',
 'cbtpa_specific_rate',
 'mfn_t

In [12]:
tariffs_df["mfn_ave_clean"] = (
    tariffs_df["mfn_ave"]
    .combine_first(tariffs_df.get("mfn_ave99"))
    .combine_first(tariffs_df.get("mfn_ave98"))
    .combine_first(tariffs_df.get("mfn_ave97"))
)

In [13]:
tariffs_df[["year", "mfn_ave", "mfn_ave97", "mfn_ave98", "mfn_ave99", "mfn_ave_clean"]].head(10)

,year,mfn_ave,mfn_ave97,mfn_ave98,mfn_ave99,mfn_ave_clean
0,1997,<NA>,0,<NA>,<NA>,0
1,1997,<NA>,0,<NA>,<NA>,0
2,1997,<NA>,0,<NA>,<NA>,0
3,1997,<NA>,0.109,<NA>,<NA>,0.109
4,1997,<NA>,0.109,<NA>,<NA>,0.109
5,1997,<NA>,0,<NA>,<NA>,0
6,1997,<NA>,0.073,<NA>,<NA>,0.073
7,1997,<NA>,0.073,<NA>,<NA>,0.073
8,1997,<NA>,0,<NA>,<NA>,0
9,1997,<NA>,0,<NA>,<NA>,0


In [14]:
# removing variables we have merged
tariffs_df = tariffs_df.drop(
    columns=[c for c in tariffs_df.columns if c.startswith("mfn_ave") and c != "mfn_ave_clean"]
)

In [15]:
[c for c in tariffs_df.columns if "colu" in c.lower() or "colo" in c.lower()]

['columbia_indicator',
 'columbia_rate_type_code',
 'columbia_ad_val_rate',
 'columbia_specific_rate',
 'columbia_other_rate',
 'colombia_indicator',
 'colombia_rate_type_code',
 'colombia_ad_val_rate',
 'colombia_specific_rate',
 'colombia_other_rate']

In [16]:
rename_map = {
    "columbia_indicator": "colombia_indicator",
    "columbia_rate_type_code": "colombia_rate_type_code",
    "columbia_ad_val_rate": "colombia_ad_val_rate",
    "columbia_specific_rate": "colombia_specific_rate",
    "columbia_other_rate": "colombia_other_rate",
}

tariffs_df = tariffs_df.rename(columns=rename_map)

In [17]:
if rebuild_tariffs_data:
    dup_names = [
        "colombia_indicator",
        "colombia_rate_type_code",
        "colombia_ad_val_rate",
        "colombia_specific_rate",
        "colombia_other_rate",
    ]

    for name in dup_names:
        cols = tariffs_df.loc[:, tariffs_df.columns == name]
        # take first value from left to right
        tariffs_df[name] = cols.bfill(axis=1).iloc[:, 0]

In [18]:
tariffs_df = tariffs_df.loc[:, ~tariffs_df.columns.duplicated()]
tariffs_df.columns.is_unique

True

In [19]:
# delete the duplicates
for base in ["indicator", "rate_type_code", "ad_val_rate", "specific_rate", "other_rate"]:
    col1 = f"colombia_{base}"
    col2 = f"columbia_{base}"

    if col1 in tariffs_df.columns and col2 in tariffs_df.columns:
        tariffs_df[col1] = tariffs_df[col1].combine_first(tariffs_df[col2])
        tariffs_df = tariffs_df.drop(columns=[col2])

In [20]:
[c for c in tariffs_df.columns if "col" in c.lower() or "colo" in c.lower()]

['col2_rate_type_code',
 'col2_ad_val_rate',
 'col2_specific_rate',
 'col2_other_rate',
 'col1_special_text',
 'col2_text_rate',
 'col1_special_mod',
 'colombia_indicator',
 'colombia_rate_type_code',
 'colombia_ad_val_rate',
 'colombia_specific_rate',
 'colombia_other_rate']

In [21]:
tariffs_df = tariffs_df.rename(columns={
    "col1_special_text": "mfn_special_text",
    "col1_special_mod": "mfn_special_modifier",
    "col2_text_rate": "non_mfn_text_rate",
    "col2_rate_type_code": "non_mfn_rate_type_code",
    "col2_ad_val_rate": "non_mfn_ad_val_rate",
    "col2_specific_rate": "non_mfn_specific_rate",
    "col2_other_rate": "non_mfn_other_rate",
})

In [22]:
tariffs_df.columns = (
    tariffs_df.columns
    .str.lower()
    .str.strip()
)

In [23]:
tariffs_df.shape, tariffs_df.columns.tolist()

((429449, 128),
 ['hts8',
  'brief_description',
  'quantity_1_code',
  'quantity_2_code',
  'mfn_rate_type_code',
  'wto_binding_code',
  'mfn_ad_val_rate',
  'mfn_specific_rate',
  'mfn_other_rate',
  'gsp_indicator',
  'gsp_ctry_excluded',
  'apta_indicator',
  'civil_air_indicator',
  'cbi_indicator',
  'cbi_ad_val_rate',
  'cbi_specific_rate',
  'israel_fta_indicator',
  'atpa_indicator',
  'atpa_ad_val_rate',
  'atpa_specific_rate',
  'pharmaceutical_ind',
  'dyes_indicator',
  'nafta_canada_ind',
  'canada_ad_val_rate',
  'canada_specific_rate',
  'canada_other_rate',
  'nafta_mexico_ind',
  'mexico_ad_val_rate',
  'mexico_specific_rate',
  'mexico_other_rate',
  'non_mfn_rate_type_code',
  'non_mfn_ad_val_rate',
  'non_mfn_specific_rate',
  'non_mfn_other_rate',
  'begin_effect_date',
  'end_effective_date',
  'year',
  'mexico_rate_type_code',
  'additional_tariff_num_ind',
  'agoa_indicator',
  'cbtpa_indicator',
  'cbtpa_rate_type_code',
  'cbtpa_ad_val_rate',
  'cbtpa_speci

##### Dtype conversions
The raw files were almost entirely read as containing strings, so we cast it to the appropriate types:
- We cast the year column we have created to `int`
- We replace `'Y'` and `'N'` with `1` and `0` in indicator columns and cast them as `int`
- We cast the rate columns as `float`
- We cast the `begin` and `end` *effective dates* columns as `datetime`

The complete and cleaned tariff dataset is saved to `data/tariffs/hs_annual_tariffs_cleaned.csv`

In [24]:
tariffs_df["year"] = pd.to_numeric(tariffs_df["year"], errors="raise")

In [25]:
indicator_cols = [
    c for c in tariffs_df if c.endswith("_indicator") or c.endswith("_ind")
]

tariffs_df[indicator_cols] = (
    tariffs_df[indicator_cols]
    .replace({"Y": 1, "N": 0})
    .apply(pd.to_numeric, errors="coerce")
    .astype("Int64")
)

In [26]:
rate_cols = [
    c for c in tariffs_df if c.endswith("_ad_val_rate") or c.endswith("_specific_rate") 
    or c.endswith("_other_rate") or c.endswith("mfn_ave") or c.endswith("mfn_ave_clean")
]

tariffs_df[rate_cols] = (
    tariffs_df[rate_cols]
    .apply(pd.to_numeric, errors="coerce")
)

In [27]:
tariffs_df["begin_effect_date"] = pd.to_datetime(
    tariffs_df["begin_effect_date"], errors="coerce"
)
tariffs_df["end_effective_date"] = pd.to_datetime(
    tariffs_df["end_effective_date"], errors="coerce"
)

##### Data cleanup
We checked that all HTS8 codes are 8 characters long, and discovered 3 were not because they were file/data drscriptors. So we removed these.

In [28]:
weird = tariffs_df[
    tariffs_df["hts8"].notna()
    & (tariffs_df["hts8"].str.len() != 8)
][["hts8", "brief_description", "year"]]

weird


,hts8,brief_description,year
213839,"Free (CA,IL,MX)",<NA>,2011
213840,"Free (CA,IL,MX); See U.S. note 3 (e)(PE)~",<NA>,2011
213841,See U.S. note 3 to this sub chapter (E); See U...,<NA>,2011


In [29]:
# check to see if all HTS codes are 8 chars long
(
    tariffs_df["hts8"]
    .dropna()
    .str.len()
    .eq(8)
    .all()
)

False

In [30]:
tariffs_df = tariffs_df[
    tariffs_df["hts8"].notna()
    & tariffs_df["hts8"].str.match(r"^\d{8}$", na=False)
].copy()

tariffs_df["hts8"].str.len().value_counts()

hts8
8    429446
Name: count, dtype: Int64

### Saving to csv

In [31]:
tariffs_df.shape

(429446, 128)

In [32]:
if rebuild_tariffs_data:
    if dataframe_sha256(tariffs_df) == '78a98c6ff065bce0ba504c57f6309e7052b42ee66bae167cbd959b54ddbe05ed':
        tariffs_df.to_csv(TARIFFS_OUT_CLEANED_CSV, index=False)
    else:
        raise ValueError("The re-wrangled tariffs data does not match the expected hash.")

In [33]:
# We then check the hash that is used in the parsing conditional statement
# dataframe_sha256(tariffs_raw_df)

## Trade data
From https://dataweb.usitc.gov the domestic exports, DOMESTIC exports, imports general and imports for consumption data was downloaded with HTS10 classification, all commodities and U.S. trade partners displayed separately and all districts and programs aggregated.

Due to the web portal's limitations the downloads had to be divided in batches by variable (dollar value, first unit quantity, second unit quantity, etc.) and, if needed, also by years.

The raw data can be consulted in `data/trade`. Each excel file has a `"Query Parameters"` sheet containing the query specifications and time of request, and a `"Query Results"` sheet containing the data.

Below, by appropriately named sections, the different files pertaining to the same trade flow are joined so that they can be then merged creating the `data/trade/merged_trade_data.csv` file.

In [34]:
PROJECT_ROOT = Path.cwd().resolve().parent

TRADE_DATA_DIR = PROJECT_ROOT / "data" / "trade"

## Verifying integrity of sources
We check:
1. we check that the intermediary merged and cleaned `csv` files exist and respect the expected hash
2. if one of the checks in `step 1` fails, we check that all raw files needed are present 
3. finally, we re-build the missing file

### Paths

In [35]:
TRADE_DATA_MERGED_CSV = TRADE_DATA_DIR / "merged_trade_data.csv"

In [36]:
IMPORTS_FOR_CONSUMPTION_DATA_DIR = TRADE_DATA_DIR / "imports_for_consumption"
IMPORTS_FOR_CONSUMPTION_RAW_DATA_DIR = IMPORTS_FOR_CONSUMPTION_DATA_DIR / "raw"

IMPORTS_FOR_CONSUMPTION_JOINED_DATA_CSV = IMPORTS_FOR_CONSUMPTION_DATA_DIR / "hts10_imports_for_consumption.csv"

IMPORTS_FOR_CONSUMPTION_DATA_DIR, IMPORTS_FOR_CONSUMPTION_JOINED_DATA_CSV

(PosixPath('/home/nicolo/Documents/uni/computer_programming_and_data_management/2/exam/assignment/data/trade/imports_for_consumption'),
 PosixPath('/home/nicolo/Documents/uni/computer_programming_and_data_management/2/exam/assignment/data/trade/imports_for_consumption/hts10_imports_for_consumption.csv'))

In [37]:
IMPORTS_GENERAL_DATA_DIR = TRADE_DATA_DIR / "imports_general"
IMPORTS_GENERAL_RAW_DATA_DIR = IMPORTS_GENERAL_DATA_DIR / "raw"

IMPORTS_GENERAL_JOINED_DATA_CSV = IMPORTS_GENERAL_DATA_DIR / "hts10_imports_general.csv"

IMPORTS_GENERAL_DATA_DIR, IMPORTS_FOR_CONSUMPTION_JOINED_DATA_CSV

(PosixPath('/home/nicolo/Documents/uni/computer_programming_and_data_management/2/exam/assignment/data/trade/imports_general'),
 PosixPath('/home/nicolo/Documents/uni/computer_programming_and_data_management/2/exam/assignment/data/trade/imports_for_consumption/hts10_imports_for_consumption.csv'))

In [38]:
FOREIGN_EXPORTS_DATA_DIR = TRADE_DATA_DIR / "foreign_exports"
FOREIGN_EXPORTS_RAW_DATA_DIR = FOREIGN_EXPORTS_DATA_DIR / "raw"

FOREIGN_EXPORTS_JOINED_DATA_CSV = FOREIGN_EXPORTS_DATA_DIR / "hts10_foreign_exports.csv"

FOREIGN_EXPORTS_DATA_DIR, FOREIGN_EXPORTS_JOINED_DATA_CSV

(PosixPath('/home/nicolo/Documents/uni/computer_programming_and_data_management/2/exam/assignment/data/trade/foreign_exports'),
 PosixPath('/home/nicolo/Documents/uni/computer_programming_and_data_management/2/exam/assignment/data/trade/foreign_exports/hts10_foreign_exports.csv'))

In [39]:
DOMESTIC_EXPORTS_DATA_DIR = TRADE_DATA_DIR / "domestic_exports"
DOMESTIC_EXPORTS_RAW_DATA_DIR = DOMESTIC_EXPORTS_DATA_DIR / "raw"

DOMESTIC_EXPORTS_JOINED_DATA_CSV = DOMESTIC_EXPORTS_DATA_DIR / "hts10_domestic_exports.csv"

DOMESTIC_EXPORTS_DATA_DIR, DOMESTIC_EXPORTS_JOINED_DATA_CSV

(PosixPath('/home/nicolo/Documents/uni/computer_programming_and_data_management/2/exam/assignment/data/trade/domestic_exports'),
 PosixPath('/home/nicolo/Documents/uni/computer_programming_and_data_management/2/exam/assignment/data/trade/domestic_exports/hts10_domestic_exports.csv'))

### Utils
Here we define a couple of functions to quickly verify the integrity of the needed files

In [40]:
def check_raw_files(raw_path, expected_raw_files: list) -> bool:
    if type(raw_path) == list:
        if [x for x in expected_raw_files in raw_path].all():
            return True
        else:
            missing = [x for x in expected_raw_files not in raw_path]
            raise ValueError(f"Could not find all needed raw files. Missing: {missing}")
    if raw_path.exists():
        if [x for x in expected_raw_files in sorted(os.listdir(raw_path))].all():
            return True
        else:
            missing = [x for x in expected_raw_files not in sorted(os.listdir(raw_path))]
            raise ValueError(f"Could not find all needed raw files. Missing: {missing}")
    else:
        raise ValueError(f"Could not find {raw_path}")

In [41]:
def verify_dataframe(csv_path, expected_hash: str):
    if csv_path.exists():
        df = pd.read_csv(csv_path, dtype="string")
        
        current_hash = dataframe_sha256(df)

        if current_hash == expected_hash:
            return df
        else:
            return None

    else:
        print(f"Could not find {csv_path}")

In [42]:
def check_integrity_data(csv_path, expected_hash: str, raw_path, expected_raw_files: list):
    df = verify_dataframe(csv_path, expected_hash)

    if df is None:
        if not check_raw_files:
            raise ValueError(f"Integrity of the raw data in {raw_path} compromised.")
        else:
            return None
    else:
        return df

### Merged trade data

In [43]:
expected_files_merged = [
    IMPORTS_FOR_CONSUMPTION_JOINED_DATA_CSV,
    IMPORTS_GENERAL_JOINED_DATA_CSV,
    FOREIGN_EXPORTS_JOINED_DATA_CSV,
    DOMESTIC_EXPORTS_JOINED_DATA_CSV
]

In [44]:
expected_trade_merged_hash = 'c9bc1936a23aa6252a53c6efe51792de098053b07432735a487db977a3dcb782'

In [45]:
trade_data_merged = check_integrity_data(
    TRADE_DATA_MERGED_CSV,
    expected_trade_merged_hash,
    TRADE_DATA_DIR,
    expected_files_merged
    )

rebuild_merged_data = (trade_data_merged is None)

### Imports for consumption

In [46]:
expected_raw_files_imports_for_consumption = [
    'hts10_calculated_duties_imports_for_consumption.xlsx',
    'hts10_customs_value_imports_for_consumption.xlsx',
    'hts10_dutiable_value_imports_for_consumption.xlsx',
    'hts10_first_quant_imports_for_consumption.xlsx',
    'hts10_landed_duty_value_imports_for_consumption.xlsx',
    'hts10_second_quant_imports_for_consumption.xlsx'
    ]

In [47]:
imports_for_consumption_hash = '6e6363166c515b81a11fcc80a55c05bfbeda9354edc90be2d01114ba531bf5fa'

In [48]:
if rebuild_merged_data:
    imports_for_consumption_df = check_integrity_data(
        IMPORTS_FOR_CONSUMPTION_JOINED_DATA_CSV,
        imports_for_consumption_hash,
        IMPORTS_FOR_CONSUMPTION_RAW_DATA_DIR,
        expected_raw_files_imports_for_consumption
        )

    rebuild_imports_for_consumption = ((imports_for_consumption_df is None) and rebuild_merged_data)
else:
    rebuild_imports_for_consumption = False

### Imports general

In [49]:
expected_raw_files_imports_general = [
    'hts10_cif_value_imports_general.xlsx',
    'hts10_customs_value_imports_general.xlsx',
    'hts10_first_quant_imports_general.xlsx',
    'hts10_general_import_charges_imports_general.xlsx',
    'hts10_second_quant_imports_general.xlsx'
    ]

In [50]:
imports_general_hash = '6e6363166c515b81a11fcc80a55c05bfbeda9354edc90be2d01114ba531bf5fa'

In [51]:
if rebuild_merged_data:
    imports_general_df = check_integrity_data(
        IMPORTS_GENERAL_JOINED_DATA_CSV,
        imports_general_hash,
        IMPORTS_GENERAL_RAW_DATA_DIR,
        expected_raw_files_imports_general
        )

    rebuild_imports_general = ((imports_general_df is None) and rebuild_merged_data)
else:
    rebuild_imports_general = False

### Foreign exports

In [52]:
expected_raw_files_foreign_exports = [
    'hts10_fas_foreign_exports.xlsx',
    'hts10_first_quant_foreign_exports.xlsx',
    'hts10_second_quant_foreign_exports.xlsx'
    ]

In [53]:
foreign_exports_hash = '5dec909357f490c1e13e6409392c8e6306dfdfc5c4484d79f6dc38c8def6f13d'

In [54]:
if rebuild_merged_data:
    foreign_exports_df = check_integrity_data(
        FOREIGN_EXPORTS_JOINED_DATA_CSV,
        foreign_exports_hash,
        FOREIGN_EXPORTS_RAW_DATA_DIR,
        expected_raw_files_foreign_exports
        )

    rebuild_foreign_exports = ((foreign_exports_df is None) and rebuild_merged_data)
else:
    rebuild_foreign_exports = False

### Domestic exports

In [55]:
expected_raw_files_domestic_exports = [
    'hts10_fas_dom_exports.xlsx',
    'hts10_first_quant_dom_exports_00-09.xlsx',
    'hts10_first_quant_dom_exports_10-24.xlsx',
    'hts10_first_quant_dom_exports_89-99.xlsx',
    'hts10_second_quant_dom_exports_10-24.xlsx',
    'hts10_second_quant_dom_exports_89-09.xlsx'
    ]

In [56]:
domestic_exports_hash = '0b981ad5470f9fe502c200d64efabc3f70f572c34eb0be77850bde46c2a10ae9'

In [57]:
if rebuild_merged_data:
    domestic_exports_df = check_integrity_data(
        DOMESTIC_EXPORTS_JOINED_DATA_CSV,
        domestic_exports_hash,
        DOMESTIC_EXPORTS_RAW_DATA_DIR,
        expected_raw_files_domestic_exports
        )

    rebuild_domestic_exports = ((domestic_exports_df is None) and rebuild_merged_data)
else:
    rebuild_domestic_exports = False

## Imports for consumption data

In [58]:
if rebuild_imports_for_consumption:
    column_to_skip = "Data Type"

    dfs = []
    failed = []

    for file in os.listdir(IMPORTS_FOR_CONSUMPTION_RAW_DATA_DIR):

        path = IMPORTS_FOR_CONSUMPTION_RAW_DATA_DIR / file

        print(f"Loading {path} ...")

        try:
            tmp_df = pd.read_excel(
                path, 
                "Query Results", 
                dtype="string", 
                skiprows=[0], 
                usecols=lambda x: x != column_to_skip
                )
            
            # strip any leading or trailing spaces in column names
            tmp_df.columns = tmp_df.columns.str.strip()

            # strip any leading or trailing spaces in key data
            key_cols = ["Country", "Year", "HTS Number", "Description"] 
            for c in key_cols:
                if c in tmp_df.columns:
                    tmp_df[c] = tmp_df[c].str.strip()

            dfs.append(tmp_df)

            if "Suppressed" in tmp_df.columns:
                print(f"It's in {file}")
        except Exception as e:
            print(f"{e} - {file}")
            failed.append(tmp_df)

    print(f"Loaded: {len(dfs)} ; failed: {len(failed)}")

In [59]:
if rebuild_imports_for_consumption:
    combined = dfs[0].copy()

    for df in dfs[1:]:
        combined = combined.merge(df, on=key_cols, how="outer", suffixes=("", "_dup"))

        dup_cols = [c for c in combined.columns if c.endswith("_dup")]
        for c in dup_cols:
            base = c[:-4]
            if base in combined.columns:
                combined[base] = combined[base].combine_first(combined[c])

            combined = combined.drop(columns=[c])

    combined

In [60]:
if rebuild_imports_for_consumption:
    imports_for_consumption_df = combined.copy()

### Data Cleanup
Only 1 row had `Year` missing. It was created by merging the "summary" rows because it is missing `Country`, `Year` and `HTS Number`.

In [61]:
# Checking that all HTS10 codes are 10 chars long
if rebuild_imports_for_consumption:
    (
        imports_for_consumption_df["HTS Number"]
        .dropna()
        .str.len()
        .eq(10)
        .all()
    )

In [62]:
# Checking that no row is missing any of the key columns' values
if rebuild_imports_for_consumption:
    print(
        imports_for_consumption_df[
        ["Country", "Year", "HTS Number"]
        ].isna().any(axis=1).sum()
        )

    print(
        (
        imports_for_consumption_df["Year"].str.strip().eq("").sum() +
        imports_for_consumption_df["Country"].str.strip().eq("").sum() +
        imports_for_consumption_df["HTS Number"].str.strip().eq("").sum()
        )
    )

### Saving to CSV
We save this "raw" merge to `data/trade/imports_for_consumption/hts10_imports_for_consumption.csv` as intermediary step to guarantee reproducibility and transparency. It also allows to avoid repeating processing steps during re-runs of the notebook.

In [63]:
if rebuild_merged_data:
    current_imports_for_consumption_hash = dataframe_sha256(imports_for_consumption_df)

    if current_imports_for_consumption_hash != imports_for_consumption_hash:
        raise ValueError("The merged trade data does not respect the expected contents.")
    else:
        if rebuild_imports_for_consumption:
            imports_for_consumption_df.to_csv(IMPORTS_FOR_CONSUMPTION_JOINED_DATA_CSV, index=False)

## Imports General

In [64]:
if rebuild_imports_general:
    column_to_skip = "Data Type"

    dfs = []
    failed = []

    for file in os.listdir(IMPORTS_GENERAL_RAW_DATA_DIR):

        path = IMPORTS_GENERAL_RAW_DATA_DIR / file

        print(f"Loading {path} ...")

        try:
            tmp_df = pd.read_excel(
                path, 
                "Query Results", 
                dtype="string", 
                skiprows=[0], 
                usecols=lambda x: x != column_to_skip
                )
            
            # strip any leading or trailing spaces in column names
            tmp_df.columns = tmp_df.columns.str.strip()

            # strip any leading or trailing spaces in key data
            key_cols = ["Country", "Year", "HTS Number", "Description"] 
            for c in key_cols:
                if c in tmp_df.columns:
                    tmp_df[c] = tmp_df[c].str.strip()

            dfs.append(tmp_df)

            if "Suppressed" in tmp_df.columns:
                print(f"It's in {file}")
        except Exception as e:
            print(f"{e} - {file}")
            failed.append(tmp_df)

    print(f"Loaded: {len(dfs)} ; failed: {len(failed)}")

In [65]:
if rebuild_imports_general:
    combined = dfs[0].copy()

    for df in dfs[1:]:
        combined = combined.merge(df, on=key_cols, how="outer", suffixes=("", "_dup"))

        dup_cols = [c for c in combined.columns if c.endswith("_dup")]
        for c in dup_cols:
            base = c[:-4]
            if base in combined.columns:
                combined[base] = combined[base].combine_first(combined[c])

            combined = combined.drop(columns=[c])

    combined

In [66]:
if rebuild_imports_general:
    imports_general_df = combined.copy()

### Data Cleanup
Only 1 row had `Year` missing. It was created by merging the "summary" rows because it is missing `Country`, `Year` and `HTS Number`.

In [67]:
# Checking that all HTS10 codes are 10 chars long
if rebuild_imports_general:
    (
        imports_general_df["HTS Number"]
        .dropna()
        .str.len()
        .eq(10)
        .all()
    )

In [68]:
# Checking that no row is missing any of the key columns' values
if rebuild_imports_general:
    print(
        imports_general_df[
        ["Country", "Year", "HTS Number"]
        ].isna().any(axis=1).sum()
        )

    print(
        (
        imports_general_df["Year"].str.strip().eq("").sum() +
        imports_general_df["Country"].str.strip().eq("").sum() +
        imports_general_df["HTS Number"].str.strip().eq("").sum()
        )
    )

### Saving to CSV
We save this "raw" merge to `data/trade/imports_general/hts10_imports_general.csv` as intermediary step to guarantee reproducibility and transparency. It also allows to avoid repeating processing steps during re-runs of the notebook.

In [69]:
if rebuild_merged_data:
    current_imports_general_hash = dataframe_sha256(imports_general_df)

    if current_imports_general_hash != imports_general_hash:
        raise ValueError("The merged trade data does not respect the expected contents.")
    else:
        if rebuild_imports_general:
            imports_general_df.to_csv(IMPORTS_GENERAL_JOINED_DATA_CSV, index=False)

## Foreign Exports

In [70]:
if rebuild_foreign_exports:
    column_to_skip = "Data Type"

    dfs = []
    failed = []

    for file in os.listdir(FOREIGN_EXPORTS_RAW_DATA_DIR):

        path = FOREIGN_EXPORTS_RAW_DATA_DIR / file

        print(f"Loading {path} ...")

        try:
            tmp_df = pd.read_excel(
                path, 
                "Query Results", 
                dtype="string", 
                skiprows=[0], 
                usecols=lambda x: x != column_to_skip
                )
            
            # strip any leading or trailing spaces in column names
            tmp_df.columns = tmp_df.columns.str.strip()

            # strip any leading or trailing spaces in key data
            key_cols = ["Country", "Year", "Schedule B", "Description"] 
            for c in key_cols:
                if c in tmp_df.columns:
                    tmp_df[c] = tmp_df[c].str.strip()

            dfs.append(tmp_df)

            if "Suppressed" in tmp_df.columns:
                print(f"It's in {file}")
        except Exception as e:
            print(f"{e} - {file}")
            failed.append(tmp_df)

    print(f"Loaded: {len(dfs)} ; failed: {len(failed)}")

In [71]:
if rebuild_foreign_exports:
    combined = dfs[0].copy()

    for df in dfs[1:]:
        combined = combined.merge(df, on=key_cols, how="outer", suffixes=("", "_dup"))

        dup_cols = [c for c in combined.columns if c.endswith("_dup")]
        for c in dup_cols:
            base = c[:-4]
            if base in combined.columns:
                combined[base] = combined[base].combine_first(combined[c])

            combined = combined.drop(columns=[c])

    combined

In [72]:
if rebuild_foreign_exports:
    foreign_exports_df = combined.copy()

### Data Cleanup
Only 1 row had `Year` missing. It was created by merging the "summary" rows because it is missing `Country`, `Year` and `Schedule B`.

In [73]:
# Checking that all HTS10 codes are 10 chars long
if rebuild_foreign_exports:
    (
        foreign_exports_df["Schedule B"]
        .dropna()
        .str.len()
        .eq(10)
        .all()
    )

In [74]:
# Checking that no row is missing any of the key columns' values
if rebuild_foreign_exports:
    print(
        foreign_exports_df[
        ["Country", "Year", "Schedule B"]
        ].isna().any(axis=1).sum()
        )

    print(
        (
        foreign_exports_df["Year"].str.strip().eq("").sum() +
        foreign_exports_df["Country"].str.strip().eq("").sum() +
        foreign_exports_df["Schedule B"].str.strip().eq("").sum()
        )
    )

### Saving to CSV
We save this "raw" merge to `data/trade/foreign_exports/hts10_foreign_exports.csv` as intermediary step to guarantee reproducibility and transparency. It also allows to avoid repeating processing steps during re-runs of the notebook.

In [75]:
if rebuild_merged_data:
    current_foreign_exports_hash = dataframe_sha256(foreign_exports_df)

    if current_foreign_exports_hash != foreign_exports_hash:
        raise ValueError("The merged trade data does not respect the expected contents.")
    else:
        if rebuild_foreign_exports:
            foreign_exports_df.to_csv(FOREIGN_EXPORTS_JOINED_DATA_CSV, index=False)

## Domestic Exports
There were some problems downloading the domestic exports data and they had thus to be downloaded in tranches, divided by years.

The wrangling of this data is thus a bit different than the others, requiring more manual concatenation before they can be merged.

In [76]:
if rebuild_domestic_exports:

    column_to_skip = "Data Type"

    fas = DOMESTIC_EXPORTS_RAW_DATA_DIR / "hts10_fas_dom_exports.xlsx"
    fq_10_24 = DOMESTIC_EXPORTS_RAW_DATA_DIR / "hts10_first_quant_dom_exports_10-24.xlsx"
    fq_00_09 = DOMESTIC_EXPORTS_RAW_DATA_DIR / "hts10_first_quant_dom_exports_00-09.xlsx"
    fq_89_99 = DOMESTIC_EXPORTS_RAW_DATA_DIR / "hts10_first_quant_dom_exports_89-99.xlsx"
    sq_10_24 = DOMESTIC_EXPORTS_RAW_DATA_DIR / "hts10_second_quant_dom_exports_10-24.xlsx"
    sq_89_09 = DOMESTIC_EXPORTS_RAW_DATA_DIR / "hts10_second_quant_dom_exports_89-09.xlsx"

    fas_df = pd.read_excel(
                fas, 
                "Query Results", 
                dtype="string", 
                skiprows=[0], 
                usecols=lambda x: x != column_to_skip
                )
    
    fq_10_24_df = pd.read_excel(
                fq_10_24, 
                "Query Results", 
                dtype="string", 
                skiprows=[0], 
                usecols=lambda x: x != column_to_skip
                )
    fq_00_09_df = pd.read_excel(
                fq_00_09, 
                "Query Results", 
                dtype="string", 
                skiprows=[0], 
                usecols=lambda x: x != column_to_skip
                )
    fq_89_99_df = pd.read_excel(
                fq_89_99, 
                "Query Results", 
                dtype="string", 
                skiprows=[0], 
                usecols=lambda x: x != column_to_skip
                )
    
    fq_df = pd.concat([fq_10_24_df, fq_00_09_df, fq_89_99_df])

    sq_10_24_df = pd.read_excel(
                sq_10_24, 
                "Query Results", 
                dtype="string", 
                skiprows=[0], 
                usecols=lambda x: x != column_to_skip
                )
    sq_89_09_df = pd.read_excel(
                sq_89_09, 
                "Query Results", 
                dtype="string", 
                skiprows=[0], 
                usecols=lambda x: x != column_to_skip
                )
    
    sq_df = pd.concat([sq_10_24_df, sq_89_09_df])

    raw_dfs = [fas_df, fq_df, sq_df]

In [77]:
dfs = []

if rebuild_domestic_exports:
    for tmp_df in raw_dfs:
        # strip any leading or trailing spaces in column names
        tmp_df.columns = tmp_df.columns.str.strip()

        # strip any leading or trailing spaces in key data
        key_cols = ["Country", "Year", "Schedule B", "Description"] 
        for c in key_cols:
            if c in tmp_df.columns:
                tmp_df[c] = tmp_df[c].str.strip()

        dfs.append(tmp_df)

print(f"Loaded: {len(dfs)}")

Loaded: 0


In [78]:
if rebuild_domestic_exports:
    combined = raw_dfs[0].copy()

    for df in raw_dfs[1:]:
        combined = combined.merge(df, on=key_cols, how="outer", suffixes=("", "_dup"))

        dup_cols = [c for c in combined.columns if c.endswith("_dup")]
        for c in dup_cols:
            base = c[:-4]
            if base in combined.columns:
                combined[base] = combined[base].combine_first(combined[c])

            combined = combined.drop(columns=[c])

    combined

In [79]:
if rebuild_domestic_exports:
    domestic_exports_df = combined.copy()

### Data Cleanup
Only 1 row had `Year` missing. It was created by merging the "summary" rows because it is missing `Country`, `Year` and `Schedule B`.

In [80]:
# Checking that all HTS10 codes are 10 chars long
if rebuild_domestic_exports:
    (
        domestic_exports_df["Schedule B"]
        .dropna()
        .str.len()
        .eq(10)
        .all()
    )

In [81]:
# Checking that no row is missing any of the key columns' values
if rebuild_domestic_exports:
    print(
        domestic_exports_df[
        ["Country", "Year", "Schedule B"]
        ].isna().any(axis=1).sum()
        )

    print(
        (
        domestic_exports_df["Year"].str.strip().eq("").sum() +
        domestic_exports_df["Country"].str.strip().eq("").sum() +
        domestic_exports_df["Schedule B"].str.strip().eq("").sum()
        )
    )

### Saving to CSV
We save this "raw" merge to `data/trade/foreign_exports/hts10_foreign_exports.csv` as intermediary step to guarantee reproducibility and transparency. It also allows to avoid repeating processing steps during re-runs of the notebook.

In [82]:
if rebuild_merged_data:
    current_domestic_exports_hash = dataframe_sha256(domestic_exports_df)

    if current_domestic_exports_hash != domestic_exports_hash:
        raise ValueError("The merged trade data does not respect the expected contents.")
    else:
        if rebuild_domestic_exports:
            domestic_exports_df.to_csv(DOMESTIC_EXPORTS_JOINED_DATA_CSV, index=False)

## Merging trade data
Even tho the data comes from the same source, some naming differences (like `HTS Number` and `Schedule B`) require column renaming.

To ensure complete transparency and clarity a prefix for each non-key row is added, expliciting which dataset it pertains to so to avoid confusion, especially with regard to duplicate names.

In [83]:
TRADE_DATA_MERGED_CSV = TRADE_DATA_DIR / "merged_trade_data.csv"

In [84]:
def standardize_columns(df):
    df = df.copy()

    # normalize column names
    df.columns = (
        df.columns
        .str.strip()
        .str.lower()
        .str.replace(" ", "_")
        .str.replace("-", "_")
    )

    # unify product code name
    df = df.rename(columns={
        "hts_number": "hts10",
        "schedule_b": "hts10",
    })

    return df


In [85]:
if rebuild_merged_data:
    imports_for_consumption_sdf = standardize_columns(imports_for_consumption_df)
    imports_general_sdf = standardize_columns(imports_general_df)
    foreign_exports_sdf = standardize_columns(foreign_exports_df)
    domestic_exports_sdf = standardize_columns(domestic_exports_df)

In [86]:
if rebuild_merged_data:
    KEYS = ["country", "year", "hts10"]

    def prefix_nonkeys(df, prefix):
        df = df.copy()
        return df.rename(
            columns={c: f"{prefix}{c}" for c in df.columns if c not in KEYS}
        )

    ifc = prefix_nonkeys(imports_for_consumption_sdf, "ifc_")
    img = prefix_nonkeys(imports_general_sdf, "img_")
    fex = prefix_nonkeys(foreign_exports_sdf, "fex_")
    dex = prefix_nonkeys(domestic_exports_sdf, "dex_")

In [87]:
if rebuild_merged_data:
    merge_keys = ["country", "year", "hts10"]

    trade_data_merged = ifc.merge(img, on=merge_keys, how="outer")
    trade_data_merged = trade_data_merged.merge(fex, on=merge_keys, how="outer")
    trade_data_merged = trade_data_merged.merge(dex, on=merge_keys, how="outer")

In [88]:
trade_data_merged.columns

Index(['country', 'year', 'hts10', 'ifc_description', 'ifc_dutiable_value',
       'ifc_customs_value', 'ifc_quantity_description',
       'ifc_second_unit_of_quantity', 'ifc_suppressed',
       'ifc_first_unit_of_quantity', 'ifc_landed_duty_paid_value',
       'ifc_calculated_duties', 'img_description', 'img_dutiable_value',
       'img_customs_value', 'img_quantity_description',
       'img_second_unit_of_quantity', 'img_suppressed',
       'img_first_unit_of_quantity', 'img_landed_duty_paid_value',
       'img_calculated_duties', 'fex_description', 'fex_quantity_description',
       'fex_first_unit_quantity', 'fex_suppressed', 'fex_second_unit_quantity',
       'fex_fas_value', 'dex_description', 'dex_fas_value',
       'dex_quantity_description', 'dex_first_unit_quantity', 'dex_suppressed',
       'dex_second_unit_quantity', 'description'],
      dtype='object')

In [89]:
# adding back description
if rebuild_merged_data:
    desc_cols = ["ifc_description", "img_description", "fex_description", "dex_description"]

    trade_data_merged["description"] = (
        trade_data_merged[desc_cols]
        .bfill(axis=1)
        .iloc[:, 0]
    )

    trade_data_merged[["description"] + desc_cols].head()

In [90]:
trade_data_merged.columns

Index(['country', 'year', 'hts10', 'ifc_description', 'ifc_dutiable_value',
       'ifc_customs_value', 'ifc_quantity_description',
       'ifc_second_unit_of_quantity', 'ifc_suppressed',
       'ifc_first_unit_of_quantity', 'ifc_landed_duty_paid_value',
       'ifc_calculated_duties', 'img_description', 'img_dutiable_value',
       'img_customs_value', 'img_quantity_description',
       'img_second_unit_of_quantity', 'img_suppressed',
       'img_first_unit_of_quantity', 'img_landed_duty_paid_value',
       'img_calculated_duties', 'fex_description', 'fex_quantity_description',
       'fex_first_unit_quantity', 'fex_suppressed', 'fex_second_unit_quantity',
       'fex_fas_value', 'dex_description', 'dex_fas_value',
       'dex_quantity_description', 'dex_first_unit_quantity', 'dex_suppressed',
       'dex_second_unit_quantity', 'description'],
      dtype='object')

In [91]:
# Checking that no row is missing any of the key columns' values
print(
    trade_data_merged[
    ["country", "year", "hts10"]
    ].isna().any(axis=1).sum()
    )

print(
    (
    trade_data_merged["year"].str.strip().eq("").sum() +
    trade_data_merged["country"].str.strip().eq("").sum() +
    trade_data_merged["hts10"].str.strip().eq("").sum()
    )
)

0
0


In [92]:
trade_data_merged.duplicated(subset=["country","year","hts10"]).sum()

0

### Saving to CSV

In [93]:
if rebuild_merged_data:
    current_trade_data_hash = dataframe_sha256(trade_data_merged)

    if current_trade_data_hash != expected_trade_merged_hash:
        raise ValueError("The merged trade data does not respect the expected contents.")
    else:
        trade_data_merged.to_csv(TRADE_DATA_MERGED_CSV, index=False)

In [94]:
trade_data_merged["ifc_suppressed"].max()

'95'

# Merging tariff and trade data

## Prepping trade data
Since the trade data was only available in hts10, 6 or 2 the requests were made with hts10, so that they could be truncated to hts8 to merge with tariffs data maintaining as much granularity as possible.

After isolating the columns to sum, we let panda's groupby handle the aggregation at the hts8 level, discarding descriptions and "suppressed" columns because there is no way to resolve their aggregation without redefinition.

In [95]:
trade_data_merged.columns

Index(['country', 'year', 'hts10', 'ifc_description', 'ifc_dutiable_value',
       'ifc_customs_value', 'ifc_quantity_description',
       'ifc_second_unit_of_quantity', 'ifc_suppressed',
       'ifc_first_unit_of_quantity', 'ifc_landed_duty_paid_value',
       'ifc_calculated_duties', 'img_description', 'img_dutiable_value',
       'img_customs_value', 'img_quantity_description',
       'img_second_unit_of_quantity', 'img_suppressed',
       'img_first_unit_of_quantity', 'img_landed_duty_paid_value',
       'img_calculated_duties', 'fex_description', 'fex_quantity_description',
       'fex_first_unit_quantity', 'fex_suppressed', 'fex_second_unit_quantity',
       'fex_fas_value', 'dex_description', 'dex_fas_value',
       'dex_quantity_description', 'dex_first_unit_quantity', 'dex_suppressed',
       'dex_second_unit_quantity', 'description'],
      dtype='object')

In [96]:
df = trade_data_merged.copy()

df["hts8"] = df["hts10"].str[:8]

sum_cols = [
    "ifc_dutiable_value","ifc_customs_value","ifc_first_unit_of_quantity",
    "ifc_second_unit_of_quantity","ifc_landed_duty_paid_value","ifc_calculated_duties",
    "img_dutiable_value","img_customs_value","img_first_unit_of_quantity",
    "img_second_unit_of_quantity","img_landed_duty_paid_value","img_calculated_duties",
    "fex_first_unit_quantity","fex_second_unit_quantity","fex_fas_value",
    "dex_first_unit_quantity","dex_second_unit_quantity","dex_fas_value",
]

flag_cols = ["ifc_suppressed","img_suppressed","fex_suppressed","dex_suppressed"]
flag_cols = [c for c in flag_cols if c in df.columns]

for c in flag_cols:
    # Turn "suppressed" into a dummy
    df[c + "_any"] = df[c].notna().astype("int8")

# Drop originals
df = df.drop(columns=flag_cols)
flag_cols = [c + "_any" for c in flag_cols]

sum_cols  = [c for c in sum_cols if c in df.columns]
flag_cols = [c for c in flag_cols if c in df.columns]

cols_needed = ["country","year","hts8"] + sum_cols + flag_cols
df = df[cols_needed]


In [97]:
agg = {c: "sum" for c in sum_cols}
for c in flag_cols:
    agg[c] = "any" # any

trade_hts8 = (
    df.groupby(["country","year","hts8"], sort=False, as_index=False)
      .agg(agg)
)

In [98]:
trade_hts8

,country,year,hts8,ifc_dutiable_value,ifc_customs_value,ifc_first_unit_of_quantity,ifc_second_unit_of_quantity,ifc_landed_duty_paid_value,ifc_calculated_duties,img_dutiable_value,...,fex_first_unit_quantity,fex_second_unit_quantity,fex_fas_value,dex_first_unit_quantity,dex_second_unit_quantity,dex_fas_value,ifc_suppressed_any,img_suppressed_any,fex_suppressed_any,dex_suppressed_any
0,Afghanistan,2020,70109005,0,5012,120,0,5497,0,0,...,0,0,0,0,0,0,True,True,False,False
1,Afghanistan,2017,85369040,0,3375,0,0,3376,0,0,...,0,0,2669,0,0,43159,True,True,True,True
2,Afghanistan,1999,41061930,975,975,6,0,1247,244,975,...,0,0,0,0,0,0,True,True,False,False
3,Afghanistan,2011,82079075,4500,4500,0,0,4742,167,4500,...,0,0,48137,00,0,3219086050,True,True,True,True
4,Afghanistan,2004,84119990,0,6945,0,0,7070,0,0,...,0,0,0,0,0,0,True,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1625697,Mexico,1993,52081300,0,0,0,0,0,0,0,...,0,0,0,0,3529,0,False,False,False,True
1625698,Mexico,1989,62079220,0,0,0,0,0,0,0,...,0,0,0,0,83,0,False,False,False,True
1625699,Mexico,2006,62103000,0,0,0,0,0,0,0,...,0,0,0,0,599,0,False,False,False,True
1625700,Mexico,1995,59032016,0,0,0,0,0,0,0,...,0,0,0,0,100612,0,False,False,False,True


## Prepping tariffs data
The tariff data was partially cleaned before, but left "untouched" in contents for transparency.

Here we remove the (hts8, year) key duplicates which can emerge via editorial or amendments reasons and then "rank" them checking:
1. If they have the `mnf_ave_clean` variable
2. If they have the `mnf_ad_val_rate` variable
3. In case of both having it, we rank by descending `begin_effective_date` to have the latest

In [99]:
list(tariffs_df.columns)

['hts8',
 'brief_description',
 'quantity_1_code',
 'quantity_2_code',
 'mfn_rate_type_code',
 'wto_binding_code',
 'mfn_ad_val_rate',
 'mfn_specific_rate',
 'mfn_other_rate',
 'gsp_indicator',
 'gsp_ctry_excluded',
 'apta_indicator',
 'civil_air_indicator',
 'cbi_indicator',
 'cbi_ad_val_rate',
 'cbi_specific_rate',
 'israel_fta_indicator',
 'atpa_indicator',
 'atpa_ad_val_rate',
 'atpa_specific_rate',
 'pharmaceutical_ind',
 'dyes_indicator',
 'nafta_canada_ind',
 'canada_ad_val_rate',
 'canada_specific_rate',
 'canada_other_rate',
 'nafta_mexico_ind',
 'mexico_ad_val_rate',
 'mexico_specific_rate',
 'mexico_other_rate',
 'non_mfn_rate_type_code',
 'non_mfn_ad_val_rate',
 'non_mfn_specific_rate',
 'non_mfn_other_rate',
 'begin_effect_date',
 'end_effective_date',
 'year',
 'mexico_rate_type_code',
 'additional_tariff_num_ind',
 'agoa_indicator',
 'cbtpa_indicator',
 'cbtpa_rate_type_code',
 'cbtpa_ad_val_rate',
 'cbtpa_specific_rate',
 'mfn_text_rate',
 'mfn_special_text',
 'atpdea_i

In [ ]:
# Check duplicates in (year, hts8) keys
dup = tariffs_df.duplicated(["year", "hts8"]).sum()
print("tariffs duplicates on (year, hts8):", dup)

tariffs duplicates on (year, hts8): 56331


In [ ]:
# Duplicate resolution
tar = tariffs_df.copy()

# Assure string dtype
tar["year"] = tar["year"].astype(str)
tar["hts8"] = tar["hts8"].astype(str)

# Cast effective date as datetime for ordering
tar["begin_effect_date"] = pd.to_datetime(tar["begin_effect_date"], errors="coerce")

# Ranking: ave, then adval and then order by most recent date
tar["_has_ave"]  = tar["mfn_ave_clean"].notna().astype(int)
tar["_has_adval"] = tar["mfn_ad_val_rate"].notna().astype(int)

tar = tar.sort_values(
    ["year","hts8","_has_ave","_has_adval","begin_effect_date"],
    ascending=[True, True, False, False, False]
)

# Drop the duplicates
tar_uniq = tar.drop_duplicates(["year","hts8"], keep="first").drop(columns=["_has_ave","_has_adval"])
print("dup after:", tar_uniq.duplicated(["year","hts8"]).sum())


dup after: 0


## Merge
After mapping the literal countries in the trade dataset to the tariff column names for the tariffs data, we create a mask so that we can filter the trade rows pertaining to each country based on the tariffs column and then `join` (not `merge`) the rates' columns on those rows.

In [108]:
TRADE_AND_TARIFFS_DATA_CSV = PROJECT_ROOT / "data" / "trade_tariffs.csv" 

In [102]:
keys = ["year", "hts8"]

tar_mfn = tar_uniq[["year","hts8","brief_description","mfn_ave_clean","mfn_ad_val_rate"]].copy()

trade1 = (trade_hts8
          .set_index(keys)
          .join(tar_mfn.set_index(keys), how="left")
          .reset_index())

In [103]:
country_to_col = {
    "Canada": "canada_ad_val_rate",
    "Mexico": "mexico_ad_val_rate",
    "Japan": "japan_ad_val_rate",
    "Korea, South": "korea_ad_val_rate",
    "Australia": "australia_ad_val_rate",
    "Chile": "chile_ad_val_rate",
    "Peru": "peru_ad_val_rate",
    "Colombia": "colombia_ad_val_rate",
    "Panama": "panama_ad_val_rate",
    "Jordan": "jordan_ad_val_rate",
    "Morocco": "morocco_ad_val_rate",
    "Singapore": "singapore_ad_val_rate",
    "Bahrain": "bahrain_ad_val_rate",
    "Oman": "oman_ad_val_rate",
}


In [104]:
all_adval = [
    c for c in tariffs_df.columns
    if c.endswith("_ad_val_rate")
    and not c.startswith(("mfn_", "non_mfn_"))
]

mapped_cols = set(country_to_col.values())

unmapped = sorted(set(all_adval) - mapped_cols)
unmapped

['atpa_ad_val_rate',
 'cbi_ad_val_rate',
 'cbtpa_ad_val_rate',
 'dr_cafta_ad_val_rate',
 'dr_cafta_plus_ad_val_rate',
 'usmca_ad_val_rate']

In [105]:
tar_rates = tar_uniq[["year","hts8"] + list(set(country_to_col.values()))].copy()
tar_rates = tar_rates.set_index(["year","hts8"])

trade1["tariff_specific_adval"] = pd.NA

for country, col in country_to_col.items():
    mask = trade1["country"].eq(country)
    if not mask.any():
        continue

    # join solo sul subset di righe di quel paese
    tmp = (trade1.loc[mask, ["year","hts8"]]
           .set_index(["year","hts8"])
           .join(tar_rates[[col]], how="left"))

    trade1.loc[mask, "tariff_specific_adval"] = tmp[col].values


In [106]:
trade1

,year,hts8,country,ifc_dutiable_value,ifc_customs_value,ifc_first_unit_of_quantity,ifc_second_unit_of_quantity,ifc_landed_duty_paid_value,ifc_calculated_duties,img_dutiable_value,...,dex_second_unit_quantity,dex_fas_value,ifc_suppressed_any,img_suppressed_any,fex_suppressed_any,dex_suppressed_any,brief_description,mfn_ave_clean,mfn_ad_val_rate,tariff_specific_adval
0,1989,01011100,Argentina,00,337334385166,1353,0,359193440664,00,00,...,0,0,True,True,False,True,<NA>,<NA>,<NA>,NaN
1,1989,01011100,Austria,0,0,0,0,0,0,0,...,0,0,False,False,False,True,<NA>,<NA>,<NA>,NaN
2,1989,01011100,Barbados,0,0,0,0,0,0,0,...,0,0,False,False,False,True,<NA>,<NA>,<NA>,NaN
3,1989,01011900,Argentina,0,1956205,740,0,2465097,0,0,...,0,17850,True,True,True,True,<NA>,<NA>,<NA>,NaN
4,1989,01011900,Australia,0,633062,124,0,850843,0,0,...,0,0,True,True,False,False,<NA>,<NA>,<NA>,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1625697,2024,99999500,Antigua and Barbuda,201016,201016,0,0,201016,0,201016,...,0,0,True,True,False,False,Estimated imports of low valued transactions,<NA>,<NA>,NaN
1625698,2024,99999500,Argentina,10168270,10168270,0,0,10168270,0,10168270,...,0,0,True,True,False,False,Estimated imports of low valued transactions,<NA>,<NA>,NaN
1625699,2024,99999500,Armenia,1407022,1407022,0,0,1407022,0,1407022,...,0,0,True,True,False,False,Estimated imports of low valued transactions,<NA>,<NA>,NaN
1625700,2024,99999500,Aruba,325720,325720,0,0,325720,0,325720,...,0,0,True,True,False,False,Estimated imports of low valued transactions,<NA>,<NA>,NaN


In [109]:
trade1.to_csv(TRADE_AND_TARIFFS_DATA_CSV, index=False)